In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project(

Given an input image in the format of some sort of file with the extension of .jpg, .png, or the likes and contains a singular cursive letter on it, we wish to be able to classify said input image in a way that'd allow us to distinguish cursive letters from one another. 

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

The data was provided to me by Mr. Rivero in the format of a google drive link, in which a large folder stored 9 folders, each one containing 26 photoes, one for each letter in cursive. This data would be then downloaded and unzipped within the jupyter notebook itself. 

The Data was then had all the .HEIC and .HEIF files converted into .JPEG in order to become readable and openable for the average human reader


In [12]:
#FUNNY ZIP :) folder_path="testData" && for file in "$folder_path"/*; do [ ${file: -4} == ".zip" ] && unzip "$file" -d "$folder_path" && echo "Unzipped: $file" || echo "Skipping non-zip file: $file"; done


#preprocessing and converting the data to a usable format
#the stinky removinator:
import shutil
# shutil.rmtree('copy')

#cleaning 
import os 
# d = 'testData'
# for f in os.listdir(d):
#     s = True
#     p = os.path.join(d, f)
#     if f.endswith('.zip'):
#         os.remove(p)
#         s = False
        
#     if s:
#         for n in os.listdir(p):
#             if 'IMG' in n:
                # os.remove(os.path.join(p, n))

#2 is an HEIF file, which I'll deal with (checked using "file [filename]"
from PIL import Image
import pillow_heif

#swap this back to temp for the whole folder
d = 'temp/6'
for f in os.listdir(d):
    #uncomment this for mass thing, this is commented only to deal with folder 2 and 6
    # for n in os.listdir(os.path.join(d, f)):
    #     p = os.path.join(os.path.join(d, f), n)
        p = os.path.join(d, f)
        # print(p)
        if (p.endswith('.JPG') == False and os.path.isdir(p) == False):
            hF = pillow_heif.read_heif(p)
            i = Image.frombytes(
                hF.mode,
                hF.size,
                hF.data.tobytes(),
                "raw",
            
            )
        
            i.save((p[:p.rfind('.')] if '.' in p else p)+".JPG", format("JPEG"))
            os.remove(p)

print('Done')

Done


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

Potential Bias: Sample size is most likely too small, as it is only 234 images, with each letter appearing only 9 times. Therefore this data is not indictative of any sort of average writing style nor is conclusive of outlier styles and/or any other unmentioned writing format. For example, a messily written m and n in cursive may be picked up as the same thing

Furthermore the data is differently lit, shot at different angles, and cropped differently, all which may affect the final outcome of the prediction as the algorithmn takes these values in. Also handwriting style for a person varies over time and therefore can be considered inconsistent and poor quality, making it quite difficult to make a model with any sort of accuracy




# 4.Prepare the Data


Apply any data transformations and explain what and why



In [1]:
#i should crop the images 

#https://www.digitalocean.com/community/tutorials/how-to-build-a-neural-network-to-recognize-handwritten-digits-with-tensorflow
#https://keras.io/examples/vision/handwriting_recognition/
import cv2, os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

#maybe I'll import a pipeline just this once because I can see the cpu inefficiency
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class midCropper(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self #this is done to make sure it easily fits into scikit learn pipe infrastructure

    def transform(self, x): #place where actual data transofrmation takes place
        #temp whatever
        imgHolder = []

        #SOMEONE DIDN'T CROP THEIR THING IT'S 8
        #screw it we ball
        for n in x:
            if not os.path.isdir(n):
                img = cv2.imread(n, cv2.IMREAD_GRAYSCALE)
                contour, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    
                if contour:
                    x1, y1, x2, y2 = cv2.boundingRect(contour[0])
                    imgHolder.append({'yV': (os.path.splitext(os.path.basename(n))[0]).lower(),
                                      #.flatten() idk
                                      'xV': (np.array(Image.fromarray(img[y1:y1 + y2, x1:x2]).resize((500, 500))).flatten())})
                    #uncomment this out if you wanna uncomment out the bottom
                    # imgHolder.append({(os.path.splitext(os.path.basename(n))[0]).lower(): Image.fromarray(img[y1:y2, x1:x2]).resize((50, 50))})
                else:
                    print('WEEE WOO WEE WOO ERROR ERROR ERROR', n)

        #only return the transformed data please :)
        return imgHolder
        
def processor(inputFolder, outputFolder):
    filePath = [os.path.join(os.path.join(inputFolder, innerFolder), fileName) for innerFolder in os.listdir(inputFolder) for fileName in os.listdir(os.path.join(inputFolder, innerFolder))] #every file path i guess

    #pipelining time
    pipeline = Pipeline([
        ('crop', midCropper()),

        #saved for when I want to fix it
        # ('processor', ColumnTransformer(transformers=[
        #     ('scale', StandardScaler(), ['xV'])
        # ],
        # remainder = 'passthrough'))
        
        #future stuf fif bug
        #future stuff is putting it in a test set lmao
    ])

    #stuff
    output = pipeline.fit_transform(filePath) #data in
    # for i, r in enumerate(output): 
    #     for fileName, imageOutput in r.items():
    #         #CHECK HERE TO JOIN OUTPUT PATH FOLDER
    #         outputPath = os.path.join(outputFolder, f"{fileName}{i}.JPG")
    #         imageOutput.save(outputPath)

    return output

#probably convert it to a vector after :)
output = processor('temp', 'test')
dF = pd.DataFrame(output)
xVM = np.array(list(dF['xV']))
xVM = xVM.reshape(xVM.shape[0], -1)
XTr, XTe, YTr, YTe = train_test_split(xVM, dF['yV'], test_size=0.15, random_state=42, stratify=dF['yV'])

print('Done')

Done


In [31]:
#stinky deleter part 2 eletric boogaloo
import shutil
# shutil.rmtree('test')

print(dF.head(), '\n\n\n', sep='')

print(XTr[0], '\n\n', YTr.head(), sep='')

#WEE EWOWO WEOWE  BUG IS THAT IT'S PROBABLY COPMRESSING thE ORIGINAL MIMAIGE NOT THE ORITHIBNG 
#@#KJ#@KJ JKLFHKJSHF SI FIX ED AND IT STiLL NO WORK OOOMOO
print('\n\n', len(dF['xV'][76]), 'aids', '\n\n')

print(XTr[0])


  yV                                                 xV
0  o  [180, 184, 183, 182, 183, 186, 188, 189, 184, ...
1  x  [192, 190, 189, 190, 192, 190, 188, 188, 191, ...
2  y  [147, 147, 150, 153, 156, 161, 169, 169, 162, ...
3  k  [176, 180, 182, 174, 160, 163, 149, 115, 119, ...
4  w  [146, 145, 147, 150, 142, 147, 165, 166, 167, ...



[234 234 234 ... 255 255 255]

164    j
16     i
156    o
177    b
161    a
Name: yV, dtype: object


 250000 aids 


[234 234 234 ... 255 255 255]


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


I made a basic neural network, as it is probably the one way to properly predict an output for a given input file. I did two methods, one using a prebuilt model using SKlean and MLPClassifier, and one by trying to build one with keras. Both are extremely bad, which is likely due to an issue with the data itself. I will finetune it just to make sure if it can be salvaged. 

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from joblib import parallel_backend

# Assuming XTr, YTr, XTe, YTe are already defined

# Create a mapping for the letters
letter_mapping = {l: i for i, l in enumerate('abcdefghijklmnopqrstuvwxyz')}

# Apply one-hot encoding
encoder = OneHotEncoder(sparse=False, categories='auto')
YTrE = encoder.fit_transform(YTr.map(letter_mapping).values.reshape(-1, 1)).astype('int64')
YTeE = encoder.transform(YTe.map(letter_mapping).values.reshape(-1, 1)).astype('int64')


# Create the MLP classifier
m1 = MLPClassifier(
    hidden_layer_sizes=(1024, 512, 256, 128, 26),
    max_iter=5000,
    learning_rate_init=0.001,
    random_state=42
)

# Use Joblib for parallel training
with parallel_backend('loky', n_jobs=80):
    m1.fit(XTr, YTrE)

# Evaluate the model
YPr = m1.predict(XTe)
a = accuracy_score(YTeE, YPr)
print(f"Accuracy: {a}")

precision, recall, f1, _ = precision_recall_fscore_support(YTeE, YPr, average='weighted', zero_division=1)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print('we have achieved liftoff')

#BROOKEN DO NOT USE, KEPT CRASHING SERVER :)


/home/jupyter-michaelw/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
#https://www.digitalocean.com/community/tutorials/how-to-build-a-neural-network-to-recognize-handwritten-digits-with-tensorflow
import tensorflow as tf
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

tf.config.threading.set_intra_op_parallelism_threads(80)
tf.config.threading.set_inter_op_parallelism_threads(80)

# map = {l: i for i, l in enumerate('abcdefghijklmnopqrstuvwxyz')}
# YTr = YTr.map(map).fillna(0).astype(np.int64)
# YTe = YTe.map(map).fillna(0).astype(np.int64)

# Create a mapping for the letters
letter_mapping = {l: i for i, l in enumerate('abcdefghijklmnopqrstuvwxyz')}

# Apply one-hot encoding
encoder = OneHotEncoder(sparse=False, categories='auto')
YTrE = encoder.fit_transform(YTr.map(letter_mapping).values.reshape(-1, 1)).astype('int64')
YTeE = encoder.transform(YTe.map(letter_mapping).values.reshape(-1, 1)).astype('int64')

print(YTr.head(), '\n\n', len(YTrE[3]), '\n\n', sep='')

#apply standard scalar later

# m = MLPClassifier(hidden_layer_sizes=(16384, 8192, 4096, 2048, 1024, 512, 256, 128, 64), max_iter=5000, learning_rate_init = 0.001, random_state=42, n_jobs=30)
# m.fit(XTr, YTrE)

strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

with strategy.scope():
    m2 = tf.keras.Sequential([
        tf.keras.layers.Dense(16384, activation='relu', input_shape=(250000,)),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(26, activation='linear')
    ])

    o = tf.keras.optimizers.Adam(learning_rate=0.001)

    m2.compile(optimizer=o,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

eS = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
m2.fit(XTr, YTrE, epochs=10, batch_size=32, validation_data=(XTe, YTeE), callbacks=[eS])

# YPr = m.predict(XTe)
# a = accuracy_score(YTeE, YPr)
# print(f"Accuracy: {a}")

# from sklearn.metrics import precision_recall_fscore_support

# precision, recall, f1, _ = precision_recall_fscore_support(YTeE, YPr, average='weighted', zero_division=1)  # Adding zero_division parameter

# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

print('we have achieved liftoff')
#---------------------------------------------------------


2023-11-17 15:08:31.609112: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 15:08:31.656180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 15:08:31.656209: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 15:08:31.656244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 15:08:31.666277: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 15:08:32.725252: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

108    w
72     h
9      z
142    e
1      x
Name: yV, dtype: object

26


Instructions for updating:
use distribute.MultiWorkerMirroredStrategy instead
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


/home/jupyter-michaelw/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10


2023-11-17 15:08:39.749302: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


...

7/7 [==============================] - 146s 21s/step - loss: 7.6280 - accuracy: 0.0354 - val_loss: 7.6280 - val_accuracy: 0.0556
Epoch 5/10
7/7 [==============================] - ETA: 0s - loss: 7.6280 - accuracy: 0.0354 

2023-11-17 15:21:00.171872: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


7/7 [==============================] - 148s 21s/step - loss: 7.6280 - accuracy: 0.0354 - val_loss: 7.6280 - val_accuracy: 0.0556
Epoch 6/10
7/7 [==============================] - ETA: 0s - loss: 7.6280 - accuracy: 0.0354 

2023-11-17 15:23:28.472207: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


7/7 [==============================] - 173s 25s/step - loss: 7.6280 - accuracy: 0.0354 - val_loss: 7.6280 - val_accuracy: 0.0556
we have achieved liftoff


In [3]:
import joblib

#joblib.dump(m1, 'm1.pkl')
joblib.dump(m2, 'm2.pkl')
print('done')

m2.save('m2.pb')
print('done')

done
INFO:tensorflow:Assets written to: m2.pb/assets


INFO:tensorflow:Assets written to: m2.pb/assets


done


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 

I attempted to finetune model2 to the best of my ability, as it's quicker to run than the MLPClassifier. Unfortunately, it's quite difficult and timeconsuming, so much improvement can not be said given the short time span (It did not improve) Furthermore, the already extremely low accuracy score is unlikely to improve anything drastic. 


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer

# Define the parameter grid
param_grid = {
    'hidden_layer_sizes': [(4096, 2048, 1024, 512, 256, 128, 64)],
    'max_iter': [10000, 5000, 1000],
    'learning_rate_init': [0.01, 0.001, 0.0001, 0.00001],
    'random_state': [42],
    'batch_size': [32, 64, 128]
}

# Create an MLPClassifier
m = MLPClassifier()

# Define a scorer (use 'neg_log_loss' if you want to optimize for log loss)
scorer = make_scorer(accuracy_score)

# Create a GridSearchCV object
grid_search = GridSearchCV(m, param_grid, scoring=scorer, cv=3, n_jobs=-1)

# Fit the model with the training data
grid_search.fit(XTr, YTrE)

# Print the best parameters and corresponding accuracy
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


1. Upon Recieving the Data, it was immediately processed by converting all the images into jpgs, grayscaling it, locating the area in which the cursive letter was to the best of my ability, and extracting that area, and then compressing and cropping that image to a preset size, namely 500x500 in my case.  
2. The model was then trained using a TensorFlow based neural network in orer to have a finer control over the neural network as well as increasing general speed in processing and training the model. 
3. Fine tuning would prove to be quite difficult and quite useless, as the error within the model itself isn't the model, or how it was constructed, but rather the poor quality data in addition to the lack of it. If given more time, as well as a bigger budget, a larger dataset would of definitely improved things. 
   

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [41]:
def something(iP):
    import tensorflow as tf
    import cv2
    import os
    import pillow_heif
    import numpy as np
    from PIL import Image
    from sklearn.model_selection import train_test_split
    import pandas as pd
    from sklearn.pipeline import Pipeline
    from sklearn.base import BaseEstimator, TransformerMixin
    
    class midCropper(BaseEstimator, TransformerMixin):
        def fit(self, x, y=None):
            return self
    
        def transform(self, x):
            imgHolder = []
            
            # def compressor(n, imgHolder):
            img = cv2.imread(x, cv2.IMREAD_GRAYSCALE)
            contour, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contour:
                x1, y1, x2, y2 = cv2.boundingRect(contour[0])
                imgHolder.append({'yV': 1,  'xV': (np.array(Image.fromarray(img[y1:y1 + y2, x1:x2]).resize((500, 500))).flatten())})
                #print('pog')
            else:
                print('WEEE WOO WEE WOO ERROR ERROR ERROR', n)
                return None

            return imgHolder
    
            # if not os.path.isdir(x):
            #     imgHolder = compressor(x, imgHolder)
            

            # return imgHolder
    
    class preProcessor(BaseEstimator, TransformerMixin):
        def fit(self, x, y=None):
            return self
    
        def transform(self, x):
            processed_images = []
    
            def r(p):
                if (p.endswith('.HEIC') == True or p.endswith('.HEIF') == True):
                    hF = pillow_heif.read_heif(p)
                    i = Image.frombytes(
                        hF.mode,
                        hF.size,
                        hF.data.tobytes(),
                        "raw",
                    )
                    processed_images.append(i)
    
                elif (p.endswith('.PNG') == True):
                    i = Image.open(p)
                    i = i.convert('RGB')
                    processed_images.append(i)
    
                elif (p.endswith('.JPG') == False and p.endswith('.JPEG') == False):
                    print('Error has occurred')
                    return None
    
            if os.path.isdir(x):
                for n, fp in zip(range(x), x):
                    r(fp)
            else:
                r(x)
    
            return processed_images

    
    def processor(input):
        if os.path.isdir(input):
            filePath = [os.path.join(input, fileName) for fileName in os.listdir(input)]
        else:
            filePath = input
    
        pipeline = Pipeline([
            #('preprocess', preProcessor()),
            ('crop', midCropper()),
        ])
    
        output = pipeline.fit_transform(filePath)
        dF = pd.DataFrame(output)
        #print(dF)
        xVM = np.array(list(dF['xV']))
        xVM = xVM.reshape(xVM.shape[0], -1)
    
        return xVM
    
    def LARP(iP, mP='m2.pb'):
        lM = tf.keras.models.load_model(mP)
    
        xVM = processor(iP)
    
        PRE = lM.predict(xVM)
    
        lM_dict = {l: i for l, i in enumerate('abcdefghijklmnopqrstuvwxyz')}
        PREP = np.array([lM_dict[np.argmax(row)] for row in PRE])
        return PREP.tolist()
    
    print(LARP(iP))

something('temp/1/a.JPG')


1/1 [==============================] - 3s 3s/step
['z']
